In [ ]:
%reload_ext google.cloud.bigquery

In [ ]:
JOB_ID = "job202404090913"

In [ ]:
# @title Example 2: Query a table with BigQuery DataFrames
# With BigQuery DataFrames, you can use many familiar Pandas methods, but the
# processing happens BigQuery rather than the runtime, allowing you to work with larger
# DataFrames that would otherwise not fit in the runtime memory.
# Learn more here: https://cloud.google.com/python/docs/reference/bigframes/latest

import bigframes.pandas as bf

bf.options.bigquery.location = "asia-southeast1" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = "curious-pointer-419406" #this variable is set based on the dataset you chose to query

In [ ]:
df = bf.read_gbq("curious-pointer-419406.raw_patents.thailand_dip_subset_1percent").to_pandas()

In [ ]:
# Define page size
page_size = 10

rows_count = len(df)
num_pages = (len(df) + page_size - 1) // page_size

print(f'Processing, Total Rows: {rows_count}, Total Pages: {num_pages}')

Processing, Total Rows: 40778, Total Pages: 4078


In [ ]:
from datetime import datetime
from google.cloud import bigquery

if JOB_ID is None:
  JOB_ID = datetime.now().strftime('job%Y%m%d%H%M')

TEMP_TABLE = f'{bf.options.bigquery.project}.processing_temp.{JOB_ID}'

print(f'Job Table is: {TEMP_TABLE}')

schema = [
    bigquery.SchemaField("ApplicationId", "INTEGER"),
    bigquery.SchemaField("Title", "STRING"),
    bigquery.SchemaField("PatentClaims", "STRING"),
]

client = bigquery.Client()
table = bigquery.Table(TEMP_TABLE, schema=schema, )
table = client.create_table(table, exists_ok=True)

Job Table is: curious-pointer-419406.processing_temp.job202404090913


In [ ]:
PROMPT = """
## Above is content from patent claims and title. Take a deep breath to consider both Title and Content then answer in this JSON format (without Markdown):
{
  "summary" : describe the patent claim in 3 sentences,
  "chemical" : if patent claim is related to chemical,
  "polymer" : if patent claim is related to polymer,
  "home_chemical" : if patent claim is related to chemical for home use,
  "industrial_chemical" : if patent claim is related to chemical for industrial use,
  "argriculture" : if patent claims is about material, product or process related to argriculture,
  "automotive" : if patent claims is about material, product or process related to automotive industry ,
  "bottle_caps" : if patent claims is about material, product or process related to production of bottle caps,
  "bottle" : if patent claims is about material, product or process related to production of bottle,
  "paint" : if patent claims is about material, product or process related to paint,
  "coating" : if patent claims is about material, product or process related to surface coating,
  "home_appliances" : if patent claims is about material, product or process related to home appliances,
  "epoxy_composite" : if patent claims is about material, product or process related to epoxy composite,
  "packaging" : if patent claims is about material, product or process related to packaging,
  "large_blow" : if patent claims is about material, product or process related to lage extrusion (large blow) products such as: barrel drum, spray tank, pontoon, bulk containers ,
  "non_woven" : if patent claims is about material, product or process related to non-woven products ,
  "pipe" : if patent claims is about material, product or process related to pipes, such as: gas pipes  liquid pipes, conduit pipes,
  "pipe_fittings" : if patent claims is about material, product or process related to pipe fittings,
  "wire" : if patent claims is about material, product or process related to wires and cables,
  "has_chemical_process" : is patent claims mentioned chemical process ,
  "has_manufacturing_process" : is patent claims mentioned manufacturing process ,
  "product" : de-duplicated array of all products and chemicals mentioned in patent claim translated to english,
}
"""

PROMPT = PROMPT.replace("\n", "")
print(PROMPT)

## Above is content from patent claims and title. Take a deep breath to consider both Title and Content then answer in this JSON format (without Markdown):{  "summary" : describe the patent claim in 3 sentences,  "chemical" : if patent claim is related to chemical,  "polymer" : if patent claim is related to polymer,  "home_chemical" : if patent claim is related to chemical for home use,  "industrial_chemical" : if patent claim is related to chemical for industrial use,  "argriculture" : if patent claims is about material, product or process related to argriculture,  "automotive" : if patent claims is about material, product or process related to automotive industry ,  "bottle_caps" : if patent claims is about material, product or process related to production of bottle caps,  "bottle" : if patent claims is about material, product or process related to production of bottle,  "paint" : if patent claims is about material, product or process related to paint,  "coating" : if patent claims 

In [ ]:
import time
from pandas_gbq import to_gbq

def bigquery(sql_query):
  query_job = client.query(sql_query)
  return query_job.result()


def get_sql():
  return f"""
  INSERT INTO patents_processed.thailand_dip_1_promptlog
    SELECT ApplicationId, Title, prompt As Prompt, ml_generate_text_llm_result As Response, ml_generate_text_status As Status, CURRENT_TIMESTAMP() as Timestamp, '{JOB_ID}' As JobId FROM
      ML.GENERATE_TEXT( MODEL `curious-pointer-419406.raw_patents.gemini`,
        ( SELECT ApplicationId, Title,
                (
                  'Title:' || Title ||
                  '\\n' ||
                  'Content:' ||
                  '\\n' ||
                  PatentClaims ||
                  '\\n\\n' ||
                  '{PROMPT}'
                )
                  as prompt
          FROM `{TEMP_TABLE}`
        ),
        STRUCT(0 AS temperature, 3 As top_k, 2000 as max_output_tokens, TRUE AS flatten_json_output));
  """

page_number = 1000
total_time = 0

for page_number in range(1000, num_pages + 1):

    bigquery(f"""
    DELETE FROM {TEMP_TABLE} WHERE true;
             """)

    page_start = (page_number - 1) * page_size
    page_end = min(page_start + page_size, rows_count)
    page_data = df.iloc[page_start:page_end]

    # Process the page data (example: print it)
    print(f" Running Page {page_number} of {num_pages}")

    start_time = time.time()

    page_data.to_gbq(TEMP_TABLE,  if_exists='replace')

    result = bigquery(get_sql())

    end_time = time.time()
    time_taken = end_time - start_time
    total_time = total_time + time_taken

    print(f"Time Taken: {total_time}, Current Pass: {time_taken}")